# CAIM Lab Session 4: Implementing search in the vector space model

In this session you will:

- Continue to work with the `arxiv` repository from last session
- Learn how to do atomic and compound search queries with ElasticSearch
- Build an inverted index for the `arxiv` repository from last session (should fit in main memory)
- Implement search in the vector space model and compare it with ElasticSearch built-in search mechanism
- Compare different implementations of search

## 1. Built-in search in ElasticSearch

ElasticSearch provides a search mechanism to make queries against a database. 
In the next code snippet you can find examples on how to do this with an atomic query (single term)
and a complex one with a so-called 'should' query (a type of OR) which admits weights in each term within the query.

In [8]:
from elasticsearch import Elasticsearch
from pprint import pprint

client = Elasticsearch("http://localhost:9200", request_timeout=1000)
client

<Elasticsearch(['http://localhost:9200'])>

#### Atomic query

In [12]:
# define query
atomic_query = {"match": {"text": "magic"}}

# search
response = client.search(index="ex3", query=atomic_query, track_total_hits=True)

# show results
# Print the results
print(f"Found {response['hits']['total']['value']} documents.")
for hit in response["hits"]["hits"][:5]:
    print(
        f"id: {hit['_id']}, score: {hit['_score']:.2f}, path: {hit['_source']['path']}, text: {hit['_source'].get('text')}"
    )

Found 72 documents.


KeyError: 'path'

#### Complex query with weights

In [3]:
# define your query with set of weighted terms
weighted_terms = {
    "search": 0.5,
    "magic": 2.0,
}

# 3. build the 'should' clauses dynamically (behaves like an OR there are other options, too)
clauses = [
    {"match": {"text": {"query": term, "boost": weight}}}  # field to search over
    for term, weight in weighted_terms.items()
]

for clause_type in ['must', 'should']:
    print()
    print(f"query type with {clause_type} clauses")

    # construct the final bool query from set of weighted terms
    es_query = {"bool": {clause_type: clauses}}

    # execute the search
    response = client.search(index="arxiv", query=es_query, track_total_hits=True)

    # Print the results
    print(f"Found {response['hits']['total']['value']} documents.")
    for hit in response["hits"]["hits"][:10]:
        print(f"id: {hit['_id']}, score: {hit['_score']:.2f}, path: {hit['_source']['path']}, text: {hit['_source'].get('text')}")


query type with must clauses


NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [arxiv]', arxiv, index_or_alias)

## 2. Excruciatingly slow search

In class we have presented a _slow_ version of search that, given a search query $q$, loops over every document in the database
computing the cosine similarity between document and query. Once this is done, it sorts documents by their similarity w.r.t. $q$ and returns the top $r$
scoring ones. 

```
1. for each d in D:
    sim(d,q) = 0
    get vector representing d
    for each w in q:
        sim(d,q) += tf(d,w) * idf(w)
    normalize sim(d,q) by |d|*|q|
2. sort results by similarity
3. return top r docs
```

A possible implementation can be found below. 

__Remark:__ _It is important to note that there are certain elements in the implementation below that refer to my own
implementation, and that you should adapt to your own; in particular, the line_

```    weights = dict(normalize(tf_idf(s['_id'])))   # gets weights as a python dict of term -> weight ```

_obtains tf-idf weights through calling a function `tf_idf` that I have implemented that, given a docid, returns a list of pairs (term, weight); and `normalize` takes such a list a normalizes weights so that the corresponding vector has length 1. 
Obviously, you should adapt the code to your own implementations from previous sessions._


In [4]:
# get tf-idf vector from doc (internal) id
def tf_idf(doc_id):
    # does nothing, adapt to your needs
    return []


# normalizes weights so that resulting vec has length 1
def normalize(l1):
    # does nothing, adapt to your needs
    return l1

In [5]:
from elasticsearch.helpers import scan
from pprint import pprint
from elasticsearch import Elasticsearch
import tqdm
import numpy as np


def preprocess_query_string(query_string, client, index_name, field_name):
    """
    given query string it outputs the list of preprocessed tokens from it
    using same analyzer (preprocessing pipeline) than the arxiv abstracts
    """

    # Use the analyze API on the specified index
    response = client.indices.analyze(
        index=index_name, field=field_name, text=query_string
    )

    # Extract just the token strings from the response
    preprocessed_terms = [token_info["token"] for token_info in response["tokens"]]

    # print(f"Original string: '{query_string}'")
    # print(f"Preprocessed terms: {preprocessed_terms}")
    return preprocessed_terms


client = Elasticsearch("http://localhost:9200", request_timeout=1000)

r = 10  # only return r top docs
# query will be list of tokens, preprocessed like the indexed arxiv articles
query_str = "searching magic"
query_tokens = preprocess_query_string(
    query_string=query_str, client=client, index_name="arxiv", field_name="text"
)

print(f"Executing search of query string '{query_str}' with tokens {query_tokens} over documents on index 'arxiv'")
sims = dict()

l2query = np.sqrt(len(query_tokens))  # l2 of query assuming 0-1 vector representation

# get nr. of docs; just for the progress bar
ndocs = int(client.cat.count(index="arxiv", format="json")[0]["count"])

# scan through docs, compute cosine sim between query and each doc
for s in tqdm.tqdm(
    scan(client, index="arxiv", query={"query": {"match_all": {}}}), total=ndocs
):

    docid = s["_source"]["path"]  # use path as id
    weights = dict(
        normalize(tf_idf(s["_id"]))
    )  # gets weights as a python dict of term -> weight (see remark above)
    sims[docid] = 0.0
    for w in query_tokens:  # gets terms as a list
        if (
            w in weights
        ):  
            sims[docid] += weights[w]  # accumulates if w in current doc
    # normalize sim
    sims[docid] /= l2query

# now sort by cosine similarity
sorted_answer = sorted(sims.items(), key=lambda kv: kv[1], reverse=True)
pprint(sorted_answer[:r])

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [arxiv]', arxiv, index_or_alias)

In [7]:
nz = len([x for x, s in sorted_answer if s > 0])
total = len(sorted_answer)
print(
    f"There are {nz} docs with non-zero similarity out of {total}, i.e. {100.0*nz/total:.1f}%"
)

There are 3492 docs with non-zero similarity out of 58102, i.e. 6.0%


## 3. Your tasks

---

**Exercise 1:**  

Make sure you understand the algorithm for implementing search described in the lecture notes. Both slow and efficient versions. Describe
the number of operations you need to do in both slow and quick versions for the following toy example with a vocabulary of size 4 and four documents:

- $q = 0,1,1,0$

- document-term matrix:
<center>


|        | t1  | t2  | t3  | t4  |
|--------|-----|-----|-----|-----|
| **d1** | 1.2 | 0.0 | 0.0 | 0.0 |
| **d2** | 0.7 | 0.3 | 1.5 | 0.1 |
| **d3** | 0.0 | 0.0 | 0.0 | 0.7 |
| **d4** | 2.0 | 0.0 | 0.0 | 0.0 |

</center>

---

**Exercise 2:**

Implement the quick version; run both slow and quick versions and report times (as a reference, in my old laptop it takes around 5m20s to run the slow version in the code above). Make sure both versions return the same answer. Note that you will need to build an inverted index in order to implement the efficient version as explained in class; it may take time but this is done once for all queries, and can be done "off-line". Also, you could improve on the code by implementing the top-$r$ sort of the final answer using
the minheap tree as discussed in class. Python has a minheap built-in implementation called `heapq`.

---

**Exercise 3:**

Compare the results for a few sample queries that you get from your quick version and ElasticSearch search. Do you get similar results? Which is faster?

---

## 4. Rules of delivery

- To be solved in _pairs_.

- No plagiarism; don't discuss your work with other teams. You can ask for help to others for simple things, such as recalling a python instruction or module, but nothing too specific to the session.

- If you feel you are spending much more time than the rest of the classmates, ask us for help. Questions can be asked either in person or by email, and you'll never be penalized by asking questions, no matter how stupid they look in retrospect.

- Write a short report listing the solutions to the exercises proposed. Include things like the important parts of your implementation (data structures used for representing objects, algorithms used, etc). You are welcome to add conclusions and findings that depart from what we asked you to do. We encourage you to discuss the difficulties you find; this lets us give you help and also improve the lab session for future editions.

- Turn the report to PDF. Make sure it has your names, date, and title. Include your code in your submission.

- Submit your work through the [raco](http://www.fib.upc.edu/en/serveis/raco.html); see date at the raco's submissions page.